In [4]:
from __future__ import absolute_import, division, print_function

from matplotlib import pyplot as plt

import tensorflow as tf
import tensorflow_hub as hub
import os

from tensorflow.keras import layers
from tensorflow.keras import optimizers 

import pandas as pd
import numpy as np

tf.VERSION

image_generator = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1/255)
train_data = image_generator.flow_from_directory("flower_photos",target_size=(32,32))
validation_data = image_generator.flow_from_directory("flower_photos_test",target_size=(32,32))

for image_batch,label_batch in train_data:
  print("Image batch shape: ", image_batch.shape)
  print("Label batch shape: ", label_batch.shape)
  break

class CollectBatchStats(tf.keras.callbacks.Callback):
  def __init__(self):
    self.batch_losses = []
    self.batch_acc = []
    self.epoch_val_loss = []
    self.epoch_val_acc = []
    
  def on_batch_end(self, batch, logs=None):
    self.batch_losses.append(logs['loss'])
    self.batch_acc.append(logs['acc'])
  def on_epoch_end(self, epoch, logs=None):
    self.epoch_val_loss.append(logs['val_loss'])
    self.epoch_val_acc.append(logs['val_acc'])

runs = [2, 3]
activation = [0, 0, 1, 1, 2, 2]
lossF = ['categorical_crossentropy', 'mean_squared_error', 'categorical_crossentropy', 'mean_squared_error', 'categorical_crossentropy', 'mean_squared_error']
saveName = ['Tanh', 'MSETanh', 'RELU', 'MSERELU', 'LeakyRELU', 'MSELeakyRELU']

Found 3303 images belonging to 5 classes.
Found 367 images belonging to 5 classes.
Image batch shape:  (32, 32, 32, 3)
Label batch shape:  (32, 5)


In [ ]:
for id in runs:
    model = None
    if (activation[id] == 0):
        # CNN 2
        model = tf.keras.Sequential([
          layers.Conv2D(32, [3, 3], (1, 1), 'valid', input_shape=(image_batch.shape[1], image_batch.shape[2], image_batch.shape[3])),
          layers.Activation('tanh'),
          # layers.LeakyReLU(alpha=0.3),
          layers.MaxPooling2D(pool_size=(2 , 2)),

          layers.Conv2D(64, [3, 3], (1, 1), 'valid'),
          layers.Activation('tanh'),
          # layers.LeakyReLU(alpha=0.3),
          layers.MaxPooling2D(pool_size=(2 , 2)),

          layers.Conv2D(128, [3, 3], (1, 1), 'valid'),
          layers.Activation('tanh'),
          # layers.LeakyReLU(alpha=0.3),
          layers.Flatten(),

          layers.Dense(train_data.num_classes, activation='softmax')
        ])
    if (activation[id] == 1):
        # CNN 2
        model = tf.keras.Sequential([
          layers.Conv2D(32, [3, 3], (1, 1), 'valid', input_shape=(image_batch.shape[1], image_batch.shape[2], image_batch.shape[3])),
          layers.Activation('relu'),
          # layers.LeakyReLU(alpha=0.3),
          layers.MaxPooling2D(pool_size=(2 , 2)),

          layers.Conv2D(64, [3, 3], (1, 1), 'valid'),
          layers.Activation('relu'),
          # layers.LeakyReLU(alpha=0.3),
          layers.MaxPooling2D(pool_size=(2 , 2)),

          layers.Conv2D(128, [3, 3], (1, 1), 'valid'),
          layers.Activation('relu'),
          # layers.LeakyReLU(alpha=0.3),
          layers.Flatten(),

          layers.Dense(train_data.num_classes, activation='softmax')
        ])
    if (activation[id] == 2):
        # CNN 2
        model = tf.keras.Sequential([
          layers.Conv2D(32, [3, 3], (1, 1), 'valid', input_shape=(image_batch.shape[1], image_batch.shape[2], image_batch.shape[3])),
          # layers.Activation('relu'),
          layers.LeakyReLU(alpha=0.3),
          layers.MaxPooling2D(pool_size=(2 , 2)),

          layers.Conv2D(64, [3, 3], (1, 1), 'valid'),
          # layers.Activation('relu'),
          layers.LeakyReLU(alpha=0.3),
          layers.MaxPooling2D(pool_size=(2 , 2)),

          layers.Conv2D(128, [3, 3], (1, 1), 'valid'),
          # layers.Activation('relu'),
          layers.LeakyReLU(alpha=0.3),
          layers.Flatten(),

          layers.Dense(train_data.num_classes, activation='softmax')
        ])
    model.compile(
        optimizer=tf.train.AdamOptimizer(), 
        loss=lossF[id],
        metrics=['accuracy'])
    
    batch_stats = CollectBatchStats()
    model.fit_generator(train_data, epochs=4, 
        callbacks = [batch_stats],
        validation_data = validation_data)
    
    results = pd.DataFrame({"losses":batch_stats.batch_losses, "accuracy":batch_stats.batch_acc})
    results.to_csv("saved_results/cnn"+saveName[id]+"Data.csv", ',')
    results = pd.DataFrame({"losses":batch_stats.epoch_val_loss, "accuracy":batch_stats.epoch_val_acc})
    results.to_csv("saved_results/cnn"+saveName[id]+"ValData.csv", ',')

In [11]:
import pandas
results = pandas.DataFrame({"losses":batch_stats.batch_losses, "accuracy":batch_stats.batch_acc})
results.to_csv("saved_results/cnn"+saveName[id]+"Data.csv", ',')
results = pandas.DataFrame({"losses":batch_stats.epoch_val_loss, "accuracy":batch_stats.epoch_val_acc})
results.to_csv("saved_results/cnn"+saveName[id]+"ValData.csv", ',')

In [12]:
from keras.datasets import cifar10
import numpy as np
import matplotlib.pyplot as plt
from collections import namedtuple as nt
from keras.utils import to_categorical

Data = nt("Data", "x_train y_train x_valid y_valid x_test y_test")
%matplotlib inline
def preprocess(data, categories):
    x_train = data.x_train.astype("float32") / 255
    x_test = data.x_test.astype("float32") / 255
    y_train = to_categorical(data.y_train, categories)
    y_test = to_categorical(data.y_test, categories)    
    return Data(x_train[5000:], y_train[5000:],
                x_train[:5000], y_train[:5000],
                x_test, y_test)
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
data = Data(x_train, y_train, None, None, x_test, y_test)

In [13]:
categories = len(np.unique(data.y_train))
processed_data = preprocess(data, categories)

In [5]:
image_generator = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1/255)

def getData(size):
    t_data = image_generator.flow_from_directory("flower_photos",target_size=(size,size))
    v_data = image_generator.flow_from_directory("flower_photos_test",target_size=(size,size))
    return t_data, v_data

def plainVanilla(images, labels, activ, los):
    print (images.shape[1], images.shape[2], images.shape[3])
    model = tf.keras.Sequential([
      layers.Flatten(input_shape=(images.shape[1], images.shape[2], images.shape[3])),
      #layers.Flatten(input_shape=(images.x_train.shape[1:])),
      layers.Dense((images.shape[1]*images.shape[2]*images.shape[3])/2, activation=activ),
      #layers.Dropout(0.2),
      layers.Dense((images.shape[1]*images.shape[2]*images.shape[3])/4, activation=activ),
      #layers.Dropout(0.2),
      #layers.Dense(128, activation=activ),
      #layers.Dropout(0.2),
      layers.Dense(labels, activation='softmax')
    ])
    
    model.compile(
        optimizer=tf.train.AdamOptimizer(), 
        loss=los,
        metrics=['accuracy'])
    return model

def trainModel(model, tr_dat, val_dat):
    batch_stats = CollectBatchStats()
    model.fit_generator(tr_dat, epochs=5, 
        callbacks = [batch_stats],         
        validation_data = val_dat)

In [6]:
i = 32
while i <= 64:
    train_dat, validation_dat = getData(i)
    for image_batch,label_batch in train_dat:
        print("Image batch shape: ", image_batch.shape)
        print("Label batch shape: ", label_batch.shape)
        break
    mlp = plainVanilla(image_batch, train_dat.num_classes, 'relu', lossF[0])
    mlp.summary()
    #mlp = plainVanilla(processed_data, categories, 'relu', lossF[0])
    trainModel(mlp, train_dat, validation_dat)
    i *= 2

Found 3303 images belonging to 5 classes.
Found 367 images belonging to 5 classes.
Image batch shape:  (32, 32, 32, 3)
Label batch shape:  (32, 5)
32 32 3
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten_1 (Flatten)          (None, 3072)              0         
_________________________________________________________________
dense_3 (Dense)              (None, 1536)              4720128   
_________________________________________________________________
dense_4 (Dense)              (None, 768)               1180416   
_________________________________________________________________
dense_5 (Dense)              (None, 5)                 3845      
Total params: 5,904,389
Trainable params: 5,904,389
Non-trainable params: 0
_________________________________________________________________
Epoch 1/5
104/104 [==============================] - 21s 204ms/step - loss: 1.8742 - acc: 0.3500 - val_loss: 

In [7]:
image_generator = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1/255)

def getData(size):
    t_data = image_generator.flow_from_directory("flower_photos",color_mode="grayscale",target_size=(size,size))
    v_data = image_generator.flow_from_directory("flower_photos_test",color_mode="grayscale",target_size=(size,size))
    return t_data, v_data

def plainVanilla(images, labels, activ, los):
    print (images.shape[1], images.shape[2], images.shape[3])
    model = tf.keras.Sequential([
      layers.Flatten(input_shape=(images.shape[1], images.shape[2], images.shape[3])),
      #layers.Flatten(input_shape=(images.x_train.shape[1:])),
      layers.Dense((images.shape[1]*images.shape[2]*images.shape[3])/2, activation=activ),
      #layers.Dropout(0.2),
      layers.Dense((images.shape[1]*images.shape[2]*images.shape[3])/4, activation=activ),
      #layers.Dropout(0.2),
      #layers.Dense(128, activation=activ),
      #layers.Dropout(0.2),
      layers.Dense(labels, activation='softmax')
    ])
    
    model.compile(
        optimizer=tf.train.AdamOptimizer(), 
        loss=los,
        metrics=['accuracy'])
    return model

def trainModel(model, tr_dat, val_dat):
    batch_stats = CollectBatchStats()
    model.fit_generator(tr_dat, epochs=5, 
        callbacks = [batch_stats],         
        validation_data = val_dat)

i = 32
while i <= 64:
    train_dat, validation_dat = getData(i)
    for image_batch,label_batch in train_dat:
        print("Image batch shape: ", image_batch.shape)
        print("Label batch shape: ", label_batch.shape)
        break
    mlp = plainVanilla(image_batch, train_dat.num_classes, 'relu', lossF[0])
    mlp.summary()
    #mlp = plainVanilla(processed_data, categories, 'relu', lossF[0])
    trainModel(mlp, train_dat, validation_dat)
    i *= 2

Found 3303 images belonging to 5 classes.
Found 367 images belonging to 5 classes.
Image batch shape:  (32, 32, 32, 1)
Label batch shape:  (32, 5)
32 32 1
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten_3 (Flatten)          (None, 1024)              0         
_________________________________________________________________
dense_9 (Dense)              (None, 512)               524800    
_________________________________________________________________
dense_10 (Dense)             (None, 256)               131328    
_________________________________________________________________
dense_11 (Dense)             (None, 5)                 1285      
Total params: 657,413
Trainable params: 657,413
Non-trainable params: 0
_________________________________________________________________
Epoch 1/5
104/104 [==============================] - 21s 202ms/step - loss: 1.5948 - acc: 0.2810 - val_loss: 1.52

In [51]:
# pak van elke soort een plaatje
soorten = os.listdir('flower_photos')[0:5]
simg = []
for n in range(5):
    simg.append(Image('flower_photos/'+soorten[n]+'/'+os.listdir('flower_photos/'+soorten[n])[5]))
print(soorten)

['roses', 'sunflowers', 'daisy', 'dandelion', 'tulips']
